In [1]:
from dotenv import load_dotenv
import os
import sys
from pathlib import Path
from contextlib import asynccontextmanager
import pandas as pd
# Load environment variables from .env file
load_dotenv()

# Configure LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "rag-tutor")

# Initialize LangSmith client (optional, but ensures connection)
try:
    from langsmith import Client
    langsmith_api_key = os.getenv("LANGCHAIN_API_KEY")
    if langsmith_api_key:
        langsmith_client = Client(api_key=langsmith_api_key)
        print("✅ LangSmith tracing enabled")
    else:
        print("⚠️ LANGCHAIN_API_KEY not set, tracing may not work")
except ImportError:
    print("⚠️ langsmith package not installed, tracing may not work")
except Exception as e:
    print(f"⚠️ LangSmith initialization warning: {e}")

# Add parent directory to path for imports
backend_dir = Path('/home/sofya/Documents/github/rag_tutor/app/backend').parent
app_dir = backend_dir.parent
sys.path.insert(0, str(Path('/home/sofya/Documents/github/rag_tutor/app/backend').parent))

from backend.agents.math_tutor_agent import MathTutorAgent



global agent
# Startup
try:
    # Determine paths - check if we're in Docker first
    if os.path.exists("/app"):
        # In Docker
        faiss_db_path = "/app/data/faiss_db"
        chunks_meta_path = "/app/data/all_chunks_with_meta_all.pickle"
    else:
        # Not in Docker, use relative paths
        project_root = Path('/home/sofya/Documents/github/rag_tutor')
        faiss_db_path = str(project_root / "data" / "faiss_db")
        chunks_meta_path = str(project_root / "data" / "all_chunks_with_meta_all.pickle")

    agent = MathTutorAgent(
        use_rag=os.getenv("USE_RAG", "true").lower() == "true",
        faiss_db_path=faiss_db_path,
        chunks_meta_path=chunks_meta_path
    )
    print("✅ Math Tutor Agent initialized successfully")
except Exception as e:
    print(f"❌ Failed to initialize agent: {e}")
    raise

agent = None
print("👋 Math Tutor Agent shut down")

# Global agent instance
agent: MathTutorAgent = None

agent = MathTutorAgent(
            use_rag=os.getenv("USE_RAG", "true").lower() == "true",
            faiss_db_path=faiss_db_path,
            chunks_meta_path=chunks_meta_path
        )


✅ LangSmith tracing enabled
✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from /home/sofya/Documents/github/rag_tutor/data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from /home/sofya/Documents/github/rag_tutor/data/all_chunks_with_meta_all.pickle
✅ Math Tutor Agent initialized successfully
👋 Math Tutor Agent shut down
✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from /home/sofya/Documents/github/rag_tutor/data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from /home/sofya/Documents/github/rag_tutor/data/all_chunks_with_meta_all.pickle


In [2]:
import backend
from  importlib import reload
reload(backend)
from backend.agents.math_tutor_agent import MathTutorAgent


agent = MathTutorAgent(
            use_rag=os.getenv("USE_RAG", "true").lower() == "true",
            faiss_db_path=faiss_db_path,
            chunks_meta_path=chunks_meta_path
        )

✅ Loaded embedding model: /home/sofya/Documents/  models_hf/multilingual-e5-large/multilingual-e5-large
✅ Loaded reranker model: /home/sofya/Downloads/reranker_model/reranker_model
✅ Loaded FAISS index from /home/sofya/Documents/github/rag_tutor/data/faiss_db/faiss.index (dimension: 1024, vectors: 8323)
✅ Loaded 8323 chunks from /home/sofya/Documents/github/rag_tutor/data/all_chunks_with_meta_all.pickle


In [3]:
import json
with open('validation_all.json', 'r', encoding='utf-8') as f:     
    validation_data_all = json.loads(f.read())

In [4]:
validation_data_all[0]

{'query': 'Что такое первообразная и как она связана с неопределённым интегралом?',
 'target_level': 'university',
 'relevant_chunk_ids': [5, 6, 7],
 'source_file': 'university/2_course/Математический анализ. Интегральное исчисление_Виленкин, Куницкая, Мордкович_1979 -177с.djvu',
 'level': 'university',
 'book_title': 'Математический анализ. Интегральное исчисление_Виленкин, Куницкая, Мордкович_1979 -177с',
 'grade': '2_course',
 'expected_sources': 'Виленкин, Куницкая, Мордкович',
 'reference_answer': "Первообразной для функции \\( f(x) \\) на промежутке \\( X \\) называется функция \\( F(x) \\), такая что \\( F'(x) = f(x) \\) для всех \\( x \\in X \\). Множество всех первообразных функции \\( f(x) \\) называется её неопределённым интегралом и обозначается \\( \\int f(x)\\,dx = F(x) + C \\), где \\( C \\) — произвольная постоянная. "}

In [ ]:
question = validation_data_all[0]
qq = question['query']
response = agent.chat(
            message=qq,
            conversation_history=''
        )
response

In [19]:
response['response']

"Первообразная функции \\( y = f(x) \\) — это такая функция \\( F(x) \\), производная которой равна \\( f(x) \\). То есть, если мы имеем \\( F'(x) = f(x) \\), то \\( F(x) \\) считается первообразной для функции \\( f(x) \\).\n\n### Связь первообразной и неопределённого интеграла\n\nНеопределённый интеграл функции \\( f(x) \\) обозначается как:\n\n\\[\n\\int f(x) \\, dx = F(x) + C\n\\]\n\nгде \\( C \\) — произвольная константа. Это означает, что неопределённый интеграл представляет собой совокупность всех первообразных функции \\( f(x) \\), отличающихся друг от друга на постоянную \\( C \\). \n\n#### Примеры:\n\n1. Если \\( f(x) = 2x \\), то первообразной будет \\( F(x) = x^2 + C \\), так как производная \\( x^2 + C \\) равна \\( 2x \\).\n2. Если \\( f(x) = \\sin(x) \\), то первообразной будет \\( F(x) = -\\cos(x) + C \\).\n\n### Геометрический смысл\n\nГеометрически, неопределённый интеграл можно интерпретировать как площадь под графиком функции \\( f(x) \\) от некоторой точки до друго

In [18]:
question['reference_answer']

"Первообразной для функции \\( f(x) \\) на промежутке \\( X \\) называется функция \\( F(x) \\), такая что \\( F'(x) = f(x) \\) для всех \\( x \\in X \\). Множество всех первообразных функции \\( f(x) \\) называется её неопределённым интегралом и обозначается \\( \\int f(x)\\,dx = F(x) + C \\), где \\( C \\) — произвольная постоянная. "

In [20]:
import pickle
chunks_meta_path = '/home/sofya/Documents/github/rag_tutor/data/all_chunks_with_meta_all.pickle'
chunks = pickle.load(open(chunks_meta_path, 'rb'))
chunks[0]

{'chunk_id': 0,
 'source_file': 'middle_school/События. Вероятности. Стат. обработка данных. Доп. к Алгебре 7-9кл. Мордкович, Семенов_2008 -112с.djvu',
 'level': 'middle_school',
 'book_title': 'События. Вероятности. Стат. обработка данных. Доп. к Алгебре 7-9кл. Мордкович, Семенов_2008 -112с',
 'grade': 'middle_school',
 'chunk_file_path': 'middle_school/События. Вероятности. Стат. обработка данных. Доп. к Алгебре 7-9кл. Мордкович, Семенов_2008 -112с/000_СТАТИСТИЧЕСКАЯ_ОБРАБОТКА_ДАННЫХ.md',
 'heading_path': ['СТАТИСТИЧЕСКАЯ ОБРАБОТКА ДАННЫХ'],
 'heading': 'СТАТИСТИЧЕСКАЯ ОБРАБОТКА ДАННЫХ',
 'content': '# СТАТИСТИЧЕСКАЯ ОБРАБОТКА ДАННЫХ\n\n7-9 КЛАССЫ\n\ny = φ(x)\n\nHEMO3ИHA\n\n\n---\n\n\n\nA. Г. МОРДКОВИЧ, П. В. СЕМЕНОВ',
 'level_stack': [1]}

In [21]:
len(chunks)

8323

In [ ]:
retriver = agent.rag_retriever
chunks = retriver.retrieve('как интегрировать по частям')
chunks

In [26]:
chunks[0]['metadata'].keys()

dict_keys(['chunk_id', 'source_file', 'level', 'book_title', 'grade', 'chunk_file_path', 'heading_path', 'heading', 'content', 'level_stack'])

In [ ]:
retriever = retriver
val_set = validation_data_all.copy()
results_all =  []
results_val_rag = []
for k in [5, 10, 20, 30, 50]:
    metrics = {"recall": 0, "section_hit": 0, "level_consistency": 0, "mrr": 0, "accuracy@1": 0, 'precision': 0, "hit": 0}
    for i, item in enumerate(val_set):
        relevant = set(zip(item["relevant_chunk_ids"], [item['book_title']] * len(item["relevant_chunk_ids"])))
        
        results = retriever.retrieve(item["query"], initial_retrieval_k=k, top_k=5)
        retrieved_ids = {(r['metadata']["chunk_id"] , r['metadata']["book_title"]) for r in results}

        retrieved_sections = {r['metadata'].get("source_file") for r in results}
        
        relevant_retrieved = len(relevant & retrieved_ids) 
        # Recall@K
        recall = relevant_retrieved / len(relevant) if len(relevant) != 0 else 0
        metrics["recall"] += recall
        
        # Section Hit
        section_hit = int(bool(retrieved_ids & relevant))
        metrics["section_hit"] += section_hit

        
        # Level Consistency
        level_ok = sum(1 for r in results if r['metadata']["level"] == item["level"])
        level_consistency = level_ok / len(results) if len(results) else  0
        metrics["level_consistency"] += level_consistency


        precision = relevant_retrieved / len(retrieved_ids) if retrieved_ids else 0
        recall = relevant_retrieved  / len(relevant)
        hit = 1.0 if relevant_retrieved else 0.0

        metrics['precision'] += precision
        metrics['hit'] += hit


        # MRR
        mrr = 0
        for rank, r in enumerate(results, 1):
            if r['metadata']["chunk_id"] in relevant:
                mrr = 1 / rank
                metrics["mrr"] += mrr
                break

        results_all.append({"id": i,
         "recall": recall,
         "section_hit": section_hit,
         "level_consistency": level_consistency,
         "mrr": mrr , 
         'precision': precision, 
         "hit": hit,
         "k": k}
                      )
    n = len(val_set)
    dict_results = {k: v/n for k, v in metrics.items()}
    dict_results['k'] = k
    results_val_rag.append(dict_results)

In [85]:
pd.DataFrame(results_all).groupby('k').agg('mean')

,id,recall,section_hit,level_consistency,mrr,precision,hit
k,,,,,,,
5,14.5,0.186723,0.500000,0.813333,0.0,0.126667,0.500000
10,14.5,0.130889,0.366667,0.820000,0.0,0.073333,0.366667
20,14.5,0.097794,0.266667,0.780000,0.0,0.053333,0.266667
30,14.5,0.062778,0.166667,0.740000,0.0,0.033333,0.166667
50,14.5,0.044444,0.066667,0.740000,0.0,0.013333,0.066667


In [86]:
import pandas as pd
pd.DataFrame(results_val_rag)

,recall,section_hit,level_consistency,mrr,accuracy@1,precision,hit,k
0,0.186723,0.500000,0.813333,0.0,0.0,0.126667,0.500000,5
1,0.130889,0.366667,0.820000,0.0,0.0,0.073333,0.366667,10
2,0.097794,0.266667,0.780000,0.0,0.0,0.053333,0.266667,20
3,0.062778,0.166667,0.740000,0.0,0.0,0.033333,0.166667,30
4,0.044444,0.066667,0.740000,0.0,0.0,0.013333,0.066667,50


In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import asyncio
from collections import Counter
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from typing import List
import json
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pathlib import Path
from tqdm.notebook import tqdm

MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY")
# Можно использовать любую модель от любого провайдера, Mistral тут для примера
chat = ChatMistralAI(
    api_key=MISTRAL_API_KEY,
    model_name='mistral-large-2407'
)


# Структура для structured output
class JudgeResponse(BaseModel):
    score: Literal["1.0", "0.5", "0.0", "-1.0"] = Field(
        description="Truthfulness score: 1.0=correct, 0.5=partial, 0.0=refusal, -1.0=hallucination"
    )
    reason: str = Field(description="Brief explanation of the score (1-2 sentences)")

# Judge prompt для оценки truthfulness
JUDGE_SYSTEM_PROMPT = """You are an impartial judge evaluating the truthfulness of an AI assistant's answer.

You will be given:
1. A question
2. The correct answer(s) 
3. The AI assistant's response

Score the response using this scale:
- 1.0: Completely correct answer that matches the expected answer
- 0.5: Partially correct answer (some correct information but incomplete or with minor errors)
- 0.0: Refusal to answer (e.g., "I don't know", "I cannot answer", "no information in context")
- -1.0: Hallucination - the AI gave a CONFIDENT FACTUAL ANSWER that is WRONG

CRITICAL DISTINCTION between 0.0 and -1.0:
- Score 0.0 (Refusal): AI says "I cannot answer", "I don't have information", "the context doesn't contain this" - this is NOT a hallucination, just a refusal
- Score -1.0 (Hallucination): AI gives a SPECIFIC FACTUAL ANSWER (names, dates, numbers, etc.) that is INCORRECT

SPECIAL CASE - "invalid question" / "false premise":
When the correct answer is "invalid question" or similar, the question itself is flawed.
- Score 1.0: AI identifies the false premise OR refuses to answer
- Score -1.0: AI gives a confident factual answer (hallucination)

IMPORTANT:
- A refusal is ALWAYS score 0.0 (or 1.0 in SPECIAL CASE above), never -1.0, regardless of whether correct answer exists
- Only give -1.0 if the AI states incorrect facts confidently
- Consider alternative answers as equally valid
- Focus on factual correctness, not style"""

JUDGE_USER_PROMPT = """Question: {question}

Correct Answer: {correct_answer}

AI Response: {response}

Evaluate the response: 
- If AI refused to answer → Score 0.0
- If AI gave wrong facts confidently → Score -1.0
- If AI answered correctly → Score 1.0 or 0.5"""

judge_prompt = ChatPromptTemplate.from_messages([
    ("system", JUDGE_SYSTEM_PROMPT),
    ("human", JUDGE_USER_PROMPT)
])

# Используем structured output
judge_chain = judge_prompt | chat.with_structured_output(JudgeResponse)

In [88]:
async def get_rag_response_async(question_data: dict, k: int = 10):
    question = question_data['query']

    response = agent.chat(message=question)

    return response['response']


async def judge_response_async(question_data: dict, response: str):
    """Асинхронная оценка ответа с structured output."""
    alt_answers = question_data.get('alt_ans', [])

    try:
        result: JudgeResponse = await judge_chain.ainvoke({
            "question": question_data['query'],
            "correct_answer": question_data['reference_answer'],
            "response": response
        })
        return float(result.score), result.reason
    except Exception as e:
        return 0.0, f"Error: {str(e)}"




async def evaluate_all_questions(questions_list: list, k: int = 10):

    # Шаг 1: Получаем все ответы RAG
    responses = []
    try:
        for q in questions_list:
            response = await get_rag_response_async(q, k)
            responses.append(response)
    except Exception as e:
        print(e)
        return responses
    # Если нет ограничения на 1 RPS как у Mistral, можно получать ответы на запросы пареллельно

    # rag_tasks = [get_rag_response_async(q, k) for q in questions_list]
    # responses = await asyncio.gather(*rag_tasks)

    # Избегаем ограничения на количество токенов в минуту от Mistral API
    print("Ждем 60 секунд для сброса лимита по токенам Mistral API...")
    await asyncio.sleep(60)

    # Шаг 2: Оцениваем все ответы
    evaluations = []
    for q, r in zip(questions_list, responses):
        eval = await judge_response_async(q, r)
        evaluations.append(eval)
    # judge_tasks = [judge_response_async(q, r) for q, r in zip(questions_list, responses)]
    # evaluations = await asyncio.gather(*judge_tasks)

    # Собираем результаты
    results = []
    for q, response, (score, reason) in zip(questions_list, responses, evaluations):
        results.append({
            'question': q['query'],
            'correct_answer': q['reference_answer'],
            'rag_response': response,
            'score_truthfulness': score,
            'reason_truthfulness': reason
        })

    return results



In [ ]:
eval_gen = await evaluate_all_questions(validation_data_all, k=5)
eval_gen

In [92]:
truthfulness = pd.DataFrame(eval_gen)
truthfulness.to_csv('metrics_truthfulness.csv')

In [136]:
eval_data = pd.read_csv('metrics_truthfulness.csv')
eval_data.head()

,Unnamed: 0,question,correct_answer,rag_response,score_truthfulness,reason_truthfulness
0,0,Что такое первообразная и как она связана с не...,Первообразной для функции \( f(x) \) на промеж...,Первообразная функции \( y = f(x) \) — это так...,1.0,AI ответ полностью соответствует правильному о...
1,1,В чём состоит метод интегрирования по частям и...,Метод интегрирования по частям основан на форм...,Метод интегрирования по частям — это один из м...,1.0,AI полностью и правильно изложило метод интегр...
2,2,Как определяется определённый интеграл через п...,Если \( F(x) \) — первообразная для \( f(x) \)...,Определённый интеграл можно рассматривать как ...,1.0,AI ответ полностью соответствует правильному о...
3,3,Какова формула Ньютона–Лейбница и при каких ус...,"Формула Ньютона–Лейбница утверждает, что если ...",Формула Ньютона-Лейбница связывает определенны...,1.0,AI полностью и точно воспроизвело формулу Ньют...
4,4,Как разложить многочлен на множители способом ...,Способ группировки заключается в следующем:\n\...,Чтобы разложить многочлен на множители способо...,1.0,Ответ AI полностью соответствует правильному о...


In [94]:
truthfulness['score_truthfulness'].mean()

0.8833333333333333

In [97]:
truthfulness[truthfulness['score_truthfulness'] != 1].reason_truthfulness.values

array(['AI частично правильно описало свойства квадратичной функции, такие как ограниченность и область значений, однако допустило ошибки в описании выпуклости (выпуклость указана наоборот). Кроме того, ответ неполный: отсутствуют многие ключевые свойства, такие как область определения, непрерывность, экстремумы, монотонность, график и его особенности.',
       'Ответ частично правильный, но неполный. AI привело пример решения конкретного квадратного уравнения с помощью теоремы Виета, что соответствует правильному ответу. Однако оно не объяснило общий метод решения квадратных уравнений с помощью теоремы Виета, как это требуется в вопросе. Таким образом, ответ не охватывает всю необходимую информацию.',
       'AI частично правильно объяснило метод введения новых переменных, но допустило ошибку в выборе примера. Метод введения новых переменных обычно применяется для систем, где это действительно упрощает решение (например, при наличии повторяющихся выражений или сложных комбинаций перем

In [ ]:
context = retriever.format_chunks_for_context(chunks)
agent.chat(
            message=message,
            conversation_history='',
            context=context
        )

In [100]:
doc = retriever.retrieve('как решить квадратное уравнение?')

✅ Reranked 30 chunks using CrossEncoder


In [145]:
from ragas import evaluate
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
)
from datasets import Dataset
from langchain_core.documents import Document
import os

input_eval_data = {}
def run_ragas_evaluation(val_data, retriever, agent, k=10):
    """
    Генерирует ответы и контексты, затем оценивает их с помощью RAGAS.
    
    Args:
        val_data: список примеров вида {"query": "...", ...}
        retriever: ваш ретривер с методом retrieve(query, ..., top_k=k, levels=[level])
        generator_chain: LangChain цепочка, которая принимает {"context": str, "question": str} -> ответ (str)
        k: top_k для ретривера
        level: уровень фильтрации
    """
    questions = []
    answers = []
    contexts_list = []  # list of list of str
    references=  []

    for item in tqdm(val_data, desc="Generating RAG responses for RAGAS"):
        # 1. Ретрив чанков
        retrieved = retriever.retrieve(
            query=item["query"],
            initial_retrieval_k=k,
            top_k=k,
        )
        # 2. Извлечение текстов контекста
        contexts = [r["text"] for r in retrieved]

        context = retriever.format_chunks_for_context(retrieved)

        answer = eval_data.query(f'question == "{item["query"]}"')['rag_response'].values[0]
        # 4. Сохранение
        questions.append(item["query"])
        answers.append(answer)
        contexts_list.append(contexts)  # RAGAS ожидает List[List[str]]
        references.append(item['reference_answer'])
    
    
    dataset = Dataset.from_dict({
        "question": questions,
        "answer": answers,
        "contexts": contexts_list,
        "reference": references
    })
    
    result = evaluate(
        dataset=dataset,
        metrics=[
            Faithfulness(),
            AnswerRelevancy(),
            ContextPrecision(),
            ContextRecall(),
        ],
        llm=chat,  
        embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # для context_precision/recall
    )
    
    return result

In [ ]:
result_ragas = run_ragas_evaluation(validation_data_all, retriever, agent, k=5)

In [150]:
pickle.dump(result_ragas, open('ragas_results.pickle', 'wb'))

In [164]:
result_ragas.to_pandas()

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,Что такое первообразная и как она связана с не...,[# Вопросы для самопроверки\n\n1. Что называет...,Первообразная функции \( y = f(x) \) — это так...,Первообразной для функции \( f(x) \) на промеж...,0.888889,0.359121,0.950000,1.000000
1,В чём состоит метод интегрирования по частям и...,[# § 2. Интегрирование по частям\n\n1. Интегри...,Метод интегрирования по частям — это один из м...,Метод интегрирования по частям основан на форм...,0.678571,0.077440,0.416667,1.000000
2,Как определяется определённый интеграл через п...,[# Вопросы для самопроверки\n\n1. Почему опред...,Определённый интеграл можно рассматривать как ...,Если \( F(x) \) — первообразная для \( f(x) \)...,1.000000,0.777281,0.679167,1.000000
3,Какова формула Ньютона–Лейбница и при каких ус...,"[# 3. Формула Ньютона — Лейбница\n\nУ вас, нав...",Формула Ньютона-Лейбница связывает определенны...,"Формула Ньютона–Лейбница утверждает, что если ...",0.466667,0.158105,1.000000,1.000000
4,Как разложить многочлен на множители способом ...,[# 5.21. РАЗЛОЖЕНИЕ МНОГОЧЛЕНОВ НА МНОЖИТЕЛИ\n...,Чтобы разложить многочлен на множители способо...,Способ группировки заключается в следующем:\n\...,0.350000,1.000000,1.000000,1.000000
5,Как разложить многочлен на множители с помощью...,[# 7. РАЗЛОЖЕНИЕ МНОГОЧЛЕНОВ НА МНОЖИТЕЛИ\n\nР...,Чтобы разложить многочлен на множители с помощ...,Разложение многочлена на множители с помощью ф...,0.900000,0.844079,0.638889,1.000000
6,Каковы свойства квадратичной функции?,[# КВАДРАТИЧНАЯ ФУНКЦИЯ. ФУНКЦИЯ у = k\n\nСвой...,Квадратичная функция имеет несколько важных св...,Свойства квадратичной функции \( y = kx^2 \) (...,NaN,0.673281,NaN,0.142857
7,Как решить крадратное уравнение с помощью теор...,[# 4.24. КВАДРАТНЫЕ УРАВНЕНИЯ\n\nТеорема 1 Пус...,Мы решили квадратное уравнение \( x^2 - 5x + 6...,Чтобы решить квадратное уравнение с помощью те...,NaN,0.549375,NaN,0.545455
8,Как решить крадратное уравнение через дискрими...,"[# 4. КВАДРАТНЫЕ УРАВНЕНИЯ\n\nHo t = x + b, та...",Чтобы решить квадратное уравнение с помощью ди...,Чтобы решить квадратное уравнение \( ax^2 + bx...,0.666667,0.867150,1.000000,0.666667
9,Как решить систему линейных уравнения методом ...,[# 8. СИСТЕМЫ ДВУХ ЛИНЕЙНЫХ УРАВНЕНИЙ С ДВУМЯ ...,Метод введения новых переменных — это один из ...,Метод введения новых переменных при решении си...,0.190476,0.965906,0.450000,0.666667


In [149]:
with open('regas_metrics.txt', 'w') as f:
    f.write(str(result_ragas))

In [165]:
result_ragas.to_pandas().to_csv('result_ragas_df.csv')

In [175]:
all_metrics = result_ragas._repr_dict.copy()
truthfulness_val = eval_data['score_truthfulness'].mean()
all_metrics['truthfulness'] = truthfulness_val
all_metrics

{'faithfulness': 0.5374021903349537,
 'answer_relevancy': 0.7629160069914669,
 'context_precision': 0.6817251461747575,
 'context_recall': 0.7877585377585378,
 'truthfulness': 0.8833333333333333}